# Cleaning

This notebook illustrates detailed download and cleaning of the Chicago Crash data. 

Our Problem focused on what factors contribute to **severe** traffic incidents at **night**. 

* **Severe** traffic incidents we defined as `FATAL` or `INCAPACITATING` from the `INJURY_TYPE` column. 

* **Night** we defined as the hours between 10pm to 5 am, or hours `22` through `5` in the `CRASH_HOUR` column. 

* Final output is `final_df` which will be used in the following notebook(s).

### Loading the Neccessary Packages and CSV Files

In [1]:
#Importing the neccessary packages
import pandas as pd
import numpy as np

In [2]:
crash_df = pd.read_csv('data/Traffic_Crashes_-_crashes.csv')
people_df = pd.read_csv('data/Traffic_Crashes_-_people.csv', low_memory=False)
vehicle_df = pd.read_csv('data/Traffic_Crashes_-_vehicles.csv', low_memory=False)

### Dropping Unneccessary Columns

#### `crash_df` dropping Justification: 

* `RD_NO` - Plice Dep. Report number, another identifying number associated with each record, we kept `CRASH_RECORD_ID` as the joining record number for each dataframe. 
* `CRASH_DATE_EST_I` - used when crash is reported to police days after the crash, this dataframe inclues crash day of week, hour and month so we can drop the specific date.
* `CRASH_DATE` - this dataframe inclues crash day of week, hour and month so we can drop the specific date.
* `REPORT_TYPE` - administrative report type, not a factor relevant to causing a crash.
* `HIT_AND_RUN_I` - not a factor relevant to causing a crash.
* `DATE_POLICE_NOTIFIED` - not a factor relevant to causing a crash.
* `STREET_NO` - of location related data we chose to keep latitude, longitude
* `BEAT_OF_OCCURENCE` - not a factor relevant to causing a crash.
* `PHOTOS_TAKEN_I` - not a factor relevant to causing a crash.
* `STATEMENTS_TAKEN` - not a factor relevant to causing a crash.
* `MOST_SEVERE_INJURY` - basing our severity of injury off of information from the `people_df` dataframe, including this and other injury related columns would cause multicolliniarity in our modeling. 
* `INJURIES_FATAL`
* `INJURIES_NON_INCAPACITATING`
* `INJURIES_REPORTED_NOT_EVIDENT`
* `INJURIES_NO_INDICATION`
* `INJURIES_UNKNOWN`

In [3]:

crash_df_dropped = crash_df[['CRASH_RECORD_ID', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 
                                          'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'LANE_CNT', 
                                          'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'NOT_RIGHT_OF_WAY_I',
                                          'HIT_AND_RUN_I', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE', 
                                          'DOORING_I','WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 
                                          'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 
                                          'LATITUDE', 'LONGITUDE']]

#### `people_df` dropping Justification: 

* `PERSON_ID` - unique ID for each person record, 

... do we need to fill in reasons for all these? 

In [4]:
people_df_dropped = people_df[['CRASH_RECORD_ID', 'PERSON_TYPE', 'AGE', 'DRIVERS_LICENSE_STATE',
                                'DRIVER_ACTION', 'DRIVER_VISION', 'BAC_RESULT VALUE', 
                                'INJURY_CLASSIFICATION']]

#### `vehicle_df` dropping Justification: 

* `---`, 

... do we need to fill in reasons for all these? 

In [5]:
vehicle_df_dropped = vehicle_df[['CRASH_RECORD_ID', 'VEHICLE_YEAR', 'MANEUVER']]

## Subsetting crash records between 10 pm and 5 am

In [6]:
night_time_df = crash_df_dropped.copy()
night_time_df = night_time_df[(night_time_df['CRASH_HOUR'] >= 22) | (night_time_df['CRASH_HOUR'] <= 5)]
night_time_df.columns

Index(['CRASH_RECORD_ID', 'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 'LANE_CNT',
       'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'NOT_RIGHT_OF_WAY_I',
       'HIT_AND_RUN_I', 'PRIM_CONTRIBUTORY_CAUSE', 'SEC_CONTRIBUTORY_CAUSE',
       'DOORING_I', 'WORK_ZONE_I', 'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I',
       'CRASH_HOUR', 'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE',
       'LONGITUDE'],
      dtype='object')

## Joining all three data sets

In [7]:
#checking the shape
night_time_df.shape, people_df.shape

((81180, 21), (1224613, 30))

In [8]:
merge1 = pd.merge(night_time_df, people_df, how='left', on='CRASH_RECORD_ID')
merge1.shape

(164919, 50)

## Further Exploring Columns

#### `INJURY_CLASSIFICATION` target Variable - this includes all people involved in incident, cyclists, passengers, drivers, etc. 

In [9]:
merge1['INJURY_CLASSIFICATION'].value_counts()

NO INDICATION OF INJURY     145253
NONINCAPACITATING INJURY     11441
REPORTED, NOT EVIDENT         4822
INCAPACITATING INJURY         2523
FATAL                          300
Name: INJURY_CLASSIFICATION, dtype: int64

In [10]:
# fatal / incapacitate = 1
merge1.loc[(merge1['INJURY_CLASSIFICATION'] == 'FATAL') | 
           (merge1['INJURY_CLASSIFICATION'] == 'INCAPACITATING INJURY'), 'INJURY_CLASSIFICATION'] = 1

# else = 0
merge1.loc[(merge1['INJURY_CLASSIFICATION'] == 'NO INDICATION OF INJURY') | 
           (merge1['INJURY_CLASSIFICATION'] == 'NONINCAPACITATING INJURY') |
           (merge1['INJURY_CLASSIFICATION'] == 'REPORTED, NOT EVIDENT'), 'INJURY_CLASSIFICATION'] = 0

merge1['INJURY_CLASSIFICATION'].fillna(0, inplace=True)

In [11]:
merge1["INJURY_CLASSIFICATION"].value_counts()

0    162096
1      2823
Name: INJURY_CLASSIFICATION, dtype: int64

In [12]:
final_df = merge1
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 164919 entries, 0 to 164918
Data columns (total 50 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   CRASH_RECORD_ID          164919 non-null  object 
 1   TRAFFIC_CONTROL_DEVICE   164919 non-null  object 
 2   DEVICE_CONDITION         164919 non-null  object 
 3   WEATHER_CONDITION        164919 non-null  object 
 4   LIGHTING_CONDITION       164919 non-null  object 
 5   LANE_CNT                 59453 non-null   float64
 6   ROADWAY_SURFACE_COND     164919 non-null  object 
 7   ROAD_DEFECT              164919 non-null  object 
 8   NOT_RIGHT_OF_WAY_I       6024 non-null    object 
 9   HIT_AND_RUN_I            65920 non-null   object 
 10  PRIM_CONTRIBUTORY_CAUSE  164919 non-null  object 
 11  SEC_CONTRIBUTORY_CAUSE   164919 non-null  object 
 12  DOORING_I                362 non-null     object 
 13  WORK_ZONE_I              929 non-null     object 
 14  WORK

#### Exporting the `final_df` into csv file

In [14]:
clean_data = final_df.to_csv('clean_data.csv', index = False)